In [41]:
import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

# load in directories
from experiment_code.constants import Defaults
from experiment_code import preprocess
from experiment_code.targetfile_utils import Utils
from experiment_code.preprocess import PilotSentences

In [42]:
pilot = PilotSentences()
df = pilot.clean_data()

In [43]:
df

,local_date,experiment_id,experiment_version,participant_public_id,participant_id,task_name,task_version,spreadsheet_version,spreadsheet_row,sentence_num,...,CoRT_std,cloze_descript,cloze_probability,dataset,random_word,target_word,word_count,trial_type,version,version_descript
312,02/06/2020 21:15:26,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),27.0,25,...,0.487950,high cloze,0.78,block_baldwin,lick,break,10.0,meaningless,4,made even distribution of cloze/CoRT across 70...
324,02/06/2020 21:15:32,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),28.0,26,...,0.500000,high cloze,0.81,peele,shook,badge,8.0,meaningless,4,made even distribution of cloze/CoRT across 70...
336,02/06/2020 21:15:38,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),29.0,27,...,1.000000,low cloze,0.38,peele,tent,rub,8.0,meaningful,4,made even distribution of cloze/CoRT across 70...
349,02/06/2020 21:15:44,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),30.0,28,...,0.836660,low cloze,0.43,peele,flip,rut,9.0,meaningful,4,made even distribution of cloze/CoRT across 70...
360,02/06/2020 21:15:49,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),31.0,29,...,1.154701,low cloze,0.41,peele,dress,trail,7.0,meaningful,4,made even distribution of cloze/CoRT across 70...
373,02/06/2020 21:15:55,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),32.0,30,...,0.836660,low cloze,0.47,peele,scared,house,9.0,meaningless,4,made even distribution of cloze/CoRT across 70...
385,02/06/2020 21:16:01,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),33.0,31,...,1.788854,high cloze,0.70,peele,gown,learn,8.0,meaningful,4,made even distribution of cloze/CoRT across 70...
398,02/06/2020 21:16:07,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),34.0,32,...,1.414214,low cloze,0.45,peele,wind,rug,9.0,meaningless,4,made even distribution of cloze/CoRT across 70...
409,02/06/2020 21:16:12,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),35.0,33,...,0.957427,low cloze,0.39,peele,joke,view,7.0,meaningless,4,made even distribution of cloze/CoRT across 70...
420,02/06/2020 21:16:18,17283.0,16.0,5e8787bf14e2970c36c20705,1487848.0,cort_prepilot_task,19.0,V4 Spreadsheet (70/30 even),36.0,34,...,1.732051,high cloze,0.70,peele,adult,dress,7.0,meaningless,4,made even distribution of cloze/CoRT across 70...


In [44]:
def _correct_blocks():
    """
    fix 'block_nums' to be sequential distribution (i.e. 1-6, not randomized) 
    Returns:
        dataframe with corrected '
    """
    blocks = df.groupby('participant_id').apply(lambda x: x.sort_values('block_num').block_num).values

    return blocks

In [45]:
df['block_num'] = _correct_blocks()

In [46]:
len(df_blocks)

5707

In [47]:
out_name = os.path.join(Defaults.RAW_DIR, f"reset_block_test.csv")
df.to_csv(out_name, header=True)

In [10]:
df_blocks = df.groupby('participant_id').apply(lambda x: x.sort_values('block_num').block_num).values

In [12]:
df['block_num']=df_blocks

In [3]:
def order_blocks(df):
    
    participants = df['participant_id'].unique()
    
    for participant in participants:
        df['block_num'] = np.sort(df['block_num'])
    
    return df

In [49]:
df['block_num']

312      1.0
324      1.0
336      1.0
349      1.0
360      1.0
        ... 
75022    6.0
75035    6.0
75047    6.0
75059    6.0
75072    6.0
Name: block_num, Length: 5707, dtype: float64

In [5]:
df['block_num'] = np.sort(df['block_num'])
#df['block_num'] = df['block_num'].apply(lambda x: sort_values(x))
df['block_num']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [12]:
df = df.sort_values(['participant_id', 'block_num']) #can't just sort
df[['block_num', 'participant_id']]

,block_num,participant_id
312,1.0,1487848.0
324,1.0,1487848.0
336,1.0,1487848.0
349,1.0,1487848.0
360,1.0,1487848.0
373,1.0,1487848.0
385,1.0,1487848.0
398,1.0,1487848.0
409,1.0,1487848.0
420,1.0,1487848.0


In [ ]:
# maybe merge with og target file???
# or include timelimit_screen

In [ ]:
df['block_num'] = df['block_num'].sort_values()
df['block_num']
# df['block_num'] = df.apply(lambda x: sorted(x['block_num']))['block_num'] 